In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import io
import scipy.misc
import numpy as np
import six
import time
import sys
import random

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
import importlib
import time
from tqdm import notebook
%matplotlib inline

In [3]:
import numpy as np
import colorsys

def _get_colors(num_colors):
    colors=[]
    for i in np.arange(0., 360., 360. / num_colors):
        hue = i/360.
        lightness = (50 + np.random.rand() * 10)/100.
        saturation = (90 + np.random.rand() * 10)/100.
        c = colorsys.hls_to_rgb(hue, lightness, saturation)
        c_t = tuple([int(255*x) for x in c])
        colors.append(c_t)
    return colors

In [8]:
os.chdir(os.path.abspath('/home/ubuntu/ibrahim/master/models/'))

In [9]:
from research.object_detection.tracking.deep_sort.deep_sort import nn_matching
from research.object_detection.tracking.deep_sort.deep_sort.detection import Detection
import research.object_detection.tracking.deep_sort.deep_sort.tracker
from research.object_detection.tracking.deep_sort.deep_sort.tracker import Tracker
from research.object_detection.tracking.deep_sort.tools import generate_detections as gdet

In [10]:
colors =_get_colors(200)
random.shuffle(colors)
def draw_sort(image, track_bbs_ids):
    image_pil=Image.fromarray(np.uint8(image)).convert('RGB')
    draw = ImageDraw.Draw(image_pil)
    im_width, im_height = image_pil.size
    for det in track_bbs_ids:
        (left, right, top, bottom) = (det[0], det[2], det[1], det[3])
        draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=2, fill=colors[det[4]])
        draw.text((left + 10, bottom -20),
                    str(det[4]),
                    fill=colors[det[4]])
    np.copyto(image, np.array(image_pil))
    
def load_image_into_numpy_array(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.asarray(image).reshape((im_height, im_width, 3)).astype(np.uint8)

# Load the COCO Label Map
category_index = {
    1: {'id': 1, 'name': 'car'},
    2: {'id': 2, 'name': 'van'},
    3: {'id': 3, 'name': 'bus'},
    4: {'id': 4, 'name': 'others'},
}

In [11]:
tf.keras.backend.clear_session()
detect_fn = tf.saved_model.load('/home/ubuntu/ibrahim/master/training/ssd_resnet/final_embeddings/saved_model/saved_model/')

2022-03-15 23:16:11.909149: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-15 23:16:15.899223: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9849 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [12]:
vidoes = [{'directory': "MVI_39211", 'type': 'easy', 'start': 0, 'end': 300 },
          {'directory': "MVI_39401", 'type': 'easy', 'start': 0, 'end': 300 },
          {'directory': "MVI_40712", 'type': 'easy', 'start': 950, 'end': 1250 },
          {'directory': "MVI_39271", 'type': 'medium', 'start': 100, 'end': 400 },
          {'directory': "MVI_40853", 'type': 'medium', 'start': 0, 'end': 300 },
          {'directory': "MVI_40772", 'type': 'medium', 'start': 800, 'end': 1100 },
          {'directory': "MVI_40901", 'type': 'hard', 'start': 0, 'end': 300 },
          {'directory': "MVI_39511", 'type': 'hard', 'start': 0, 'end': 300 },
          {'directory': "MVI_39511", 'type': 'hard', 'start': 0, 'end': 300 }]
DETRAC_TEST = '/home/ubuntu/ibrahim/master/datasets/Insight-MVT_Annotation_Test'

In [13]:
vidoes = [{'directory': "MVI_39271", 'type': 'medium', 'start': 100, 'end': 400 }]

In [14]:
th = 0.3
for ent in notebook.tqdm(vidoes):
    directory, _type, start, end = ent['directory'],ent['type'],ent['start'],ent['end']
    path_write = f"/home/ubuntu/ibrahim/master/results/retina_final_embeddings/{_type}/{directory}"
    if not os.path.exists(path_write):
        os.makedirs(path_write)
    counter = 0
    if os.path.isdir(os.path.join(DETRAC_TEST, directory)):
        importlib.reload(research.object_detection.tracking.deep_sort.deep_sort.tracker)
        deep_tracker = Tracker(nn_matching.NearestNeighborDistanceMetric("cosine", 0.3, None))
        for image in notebook.tqdm(sorted(os.listdir(os.path.join(DETRAC_TEST, directory)))):
            counter += 1
            if counter < start or counter > end:
                continue
            image_path = os.path.join(DETRAC_TEST, directory, image)
            frame_no = int(image.replace('img', '').replace('.jpg', ''))
            image_np = load_image_into_numpy_array(image_path)
            height, width, _ = image_np.shape
            input_tensor = np.expand_dims(image_np, 0)
            detections = detect_fn(input_tensor)
            boxes = detections['detection_boxes'][0].numpy()
            classes = detections['detection_classes'][0].numpy().astype(np.int32)
            scores = detections['detection_scores'][0].numpy()
            emb = detections['nms_embedding'][0].numpy()
            det = []
            features = []
            for f in range(boxes.shape[0]):
                box = boxes[f]
                if scores[f] > th:
                    box_top = int(box[0] * height)
                    box_left = int(box[1] * width)
                    box_bottom = int(box[2] * height)
                    box_right = int(box[3] * width)
                    det.append([box_left, box_top, (box_right-box_left), (box_bottom-box_top), scores[f], classes[f]])
                    features.append(emb[f])
            detections_deep = [Detection((bbox[0], bbox[1], bbox[2], bbox[3]), bbox[4], feature)
                                    for bbox, feature in zip(det, features)]
            deep_tracker.predict()
            deep_tracker.update(detections_deep)
            track_bbs_ids = []
            for track in deep_tracker.tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue
                track_bbs_ids.append([*track.to_tlbr(), track.track_id])
            image_np_with_detections = image_np.copy()
            draw_sort(image_np_with_detections, track_bbs_ids)
            if counter > start and counter < end and counter%25 == 0:
                im = Image.fromarray(image_np_with_detections)
                im.save(f"{path_write}/{directory}_{counter}.jpg")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1570 [00:00<?, ?it/s]

2022-03-15 23:17:30.329206: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8302
2022-03-15 23:17:33.589947: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
